# Create and Write SPECFEM3D DATA Files: CMT, STATIONS, ADJ_STATIONS 
1. Define function for making directories
1. Create directories for each event including the special directories for run0001 

### 1. Unpickle Project Dictionary

In [ ]:
import pickle

data_root_dir = '../../../data_notebooks'

# Test UnPickle Project Dictionary
f = open(data_root_dir + '/pickled/project_dict.pickle', 'rb')
dill_project_dict = pickle.load(f)
f.close()
print()
print('dill_project_dict:\n',dill_project_dict)
print()

### 2. Unpickle events to be used for constructing moment tensors

In [ ]:
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

#Unpickle events
print('Unpickling Events')
f = open('../../../data_notebooks/pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open('../../../data_notebooks/pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

### 3. Construct moment tensors

In [ ]:
from gnam.events.mtensors import mtensors
import pandas as pd
import numpy as np

#Get model bounding box needed for adjusting strike
gf_bbox = dill_events.getBBox()

#Read moment tensors
gf_mts = mtensors(data_root_dir + '/data/event_moments.csv',gf_bbox)

    
# get event catalog of the events withing the bounding box
e_cat = dill_events.getIncCatalog()

# This is a bit hokey, but it works. Here we update the
# event time from the moment tensor CSV file with thouse
# from the event catalog
gf_mts.update_utcdatetime(e_cat)
f.close()

# Create a dictionary that maps moment tensors to events
e2mt_dict = gf_mts.map_events_2_tensors(e_cat)
e2mt_keys = e2mt_dict.keys()

# Print a comparison of events to moment tensors
for key in e2mt_keys:
    print('UTC: event[%d][Date] = %s' %(key,e_cat[key].origins[0].time))
    print('UTC:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['Date']))
    print('Mag: event[%d][Date] = %s' %(key,e_cat[key].magnitudes[0].mag))
    print('Mag:    MT[%d][Date] = %s' %(key,e2mt_dict[key]['ML']))
    print('Mag:    MT[%d][XC]   = %s' %(key,e2mt_dict[key]['XC']))
    print('Mag:    MT[%d][YC]   = %s' %(key,e2mt_dict[key]['YC']))
    print('Dict: %d' %(key),e2mt_dict[key])
    print('Obj:  %d' %(key),gf_mts[3])
    print()

### 4. Unpickle Noisey (removed) and Kept Streams

In [ ]:
# unpickle kept streams
f = open(data_root_dir + '/pickled/kept_obs_data.pickle', 'rb')
dill_kept_obs_dict = pickle.load(f)
f.close()

dill_obs_keys = dill_kept_obs_dict.keys()
for dkey in dill_obs_keys:
    print(len(dill_kept_obs_dict[dkey]))

# unpickle removed streams
f = open(data_root_dir + '/pickled/remo_obs_data.pickle', 'rb')
dill_remo_obs_dict = pickle.load(f)
f.close()

dill_obs_keys = dill_remo_obs_dict.keys()
for dkey in dill_obs_keys:
    print(len(dill_remo_obs_dict[dkey]))

### 5. Unpickle Event trigger dictionary

In [ ]:
#test unpickle the event_trigger_dict[eid]
f = open(data_root_dir + '/pickled/event_trigger_dict.pickle', 'rb')
event_trigger_dict = pickle.load(f)
f.close()

trig_keys = event_trigger_dict.keys()
for tkey in trig_keys:
    print('tkey: %d\n'%(tkey),event_trigger_dict[tkey])
    print()

### 6. CMTSOLUTION, STATIONS, STATIONS_ADJOINT, trace, and .window files for each event (run#### dirs)

#### Define function for writing .window files used for creating adjoint sources

In [ ]:
def write_window_files(fqdp,trig_dict,except_st,win=1.5):
    
    for key in trig_dict:
        
        #FIXME: ugly ugly ugly
        skip = False
        for tr in except_st:
            if key == tr.stats.station:
                skip = True
                break
        if skip: 
            continue
        
        tr_filename = 'NL.' + key + '.window'
        
        p_trig = trig_dict[key][0]
        p0     = trig_dict[key][1]
        s0     = trig_dict[key][2]
        s_trig = trig_dict[key][3]
        str_t0 = 'T0 = %.3f\n' %(p_trig)
        str_t1 = 'T1 = %.3f\n' %(p0+win)
        str_p0 = 'P0 = %.3f\n' %(p0)
        str_s0 = 'S0 = %.3f\n' %(s0)
        #print('T0:',str_t0)
        #print('T1:',str_t1)
        window_list = [str_t0,str_t1,str_p0,str_s0] 

        #print('Filename:', tr_filename)
        #print('Contents:\n', window_list)

        fqpname = fqdp + '/' + tr_filename
        f = open(fqpname, 'w')
        f.writelines(window_list)
        f.close()

In [ ]:
from gnam.specutils.write import write_cmtsolution as spec_wcmt
from gnam.specutils.write import write_stations_except_for as spec_ws_e4
from gnam.specutils.write import write_stream_2_spec_ascii as spec_wst2a
from shutil import copyfile

num_events = dill_project_dict['NUM_EVENTS']
proj_name = dill_project_dict['PROJ_NAME']
proj_root_path = dill_project_dict['PROJ_ROOT_PATH']
proj_fqdn = proj_root_path + proj_name

for e in range(num_events):
    ekey = e + 3 #FIXME!!!
    edir = proj_fqdn + '/run' + str(e+1).zfill(4)
    dname = edir
    data_dir = dname + '/Data'
    obs_dir = dname + '/OBS'
    
    #Copy a starting Parfile
    src = data_root_dir + '/SPECFEM3D' + '/Par_file_event0'
    if e != 0:
        src = data_root_dir + '/SPECFEM3D' + '/Par_file_eventN'
        
    dst = data_dir + '/Par_file'
    copyfile(src, dst)
    
    #needed for excluding sations in STATION files and OBS data
    except_st = dill_remo_obs_dict[ekey]
    
    #needed for .window files used for creating adj-src's
    trig_dict = event_trigger_dict[ekey]
    
    #FIXME!!! used for geting stations for writing STATION Files
    bkeys = [3,4]
    
    print('Writing for Experiment-%d:\n'%(ekey), dname)
    
    #write moment tensors
    mt = e2mt_dict[ekey] 
    xc = e2mt_dict[ekey]['XC']
    yc = e2mt_dict[ekey]['YC']
    lxy = gf_bbox.convert_extern_coords_2_local(xc,yc)
    spec_wcmt(data_dir + '/CMTSOLUTION',mt,lxy[0],lxy[1])
    
    #write STATIONS
    mt = e2mt_dict[ekey] 
    spec_ws_e4(data_dir + '/STATIONS',dill_straces,ekey,bkeys,except_st)
    
    src = data_dir + '/STATIONS'
    dst = data_dir + '/STATIONS_ADJOINT'
    copyfile(src, dst)
    #spec_ws_e4(data_dir + '/STATIONS_ADJOINT',dill_straces,ekey,bkeys,except_st)
    
    #write OBS data (the actual traces)
    obs_st = dill_kept_obs_dict[ekey]
    spec_wst2a(obs_st,obs_dir,'FX#')
    
    #write .window files 
    write_window_files(obs_dir,trig_dict,except_st)

## Finished